In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.preprocessing import MultiLabelBinarizer
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from tensorflow.keras.applications import InceptionResNetV2
import urllib.request

pd.set_option("display.max_columns", None)

2023-05-14 13:44:12.175333: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-14 13:44:12.200162: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-14 13:44:12.331034: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-14 13:44:12.333457: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 13:44:13.335887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
image_size = 75

In [3]:
train = pd.read_csv('train.csv', header=None, names=["image", "labels"])
print("Train ammount:",len(train))
print("Train columns:", train.columns)
print(train['labels'].value_counts())

Train ammount: 64
Train columns: Index(['image', 'labels'], dtype='object')
TwistedFate    1
Garen          1
Ahri           1
Orianna        1
Tryndamere     1
              ..
Fiora          1
Tristana       1
Diana          1
DrMundo        1
Kaisa          1
Name: labels, Length: 64, dtype: int64


In [4]:
# %%time
# datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
#                                                         preprocessing_function=None,
#                                                         data_format=None,
#                                                     )

# train_data = datagen.flow_from_dataframe(
#     train,
#     directory='./train_dataset',
# #     directory='./train_dataset_round',
#     x_col="image",
#     y_col= 'labels',
#     color_mode="rgb",
#     target_size = (256,256),
#     class_mode="categorical",
#     batch_size=32,
#     shuffle=False,
#     seed=40,
# )

datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_data = datagen.flow_from_dataframe(
    train,
    directory='./train_dataset',
#     directory='./train_dataset_round',
    x_col="image",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (image_size,image_size),
    class_mode="categorical",
    batch_size=16,
    shuffle=False,
    seed=40,
)

Found 64 validated image filenames belonging to 64 classes.


In [5]:
seed = 1200
tf.random.set_seed(seed)
num_classes = 64

# url = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.7/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'

# weights_path = './keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'

url = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.7/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5'

weights_path = './keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5'

if not os.path.exists(weights_path):
    urllib.request.urlretrieve(url, weights_path)
    
# Load the pre-trained InceptionResNetV2 model
base_model = keras.applications.InceptionResNetV2(weights=weights_path, include_top=True, input_shape=(image_size, image_size, 3))

# Remove the last layer of the model
x = base_model.layers[-2].output

# Add a new fully connected layer with 64 output neurons
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, activation='relu')(x)
output = keras.layers.Dense(num_classes, activation='softmax')(x)

# Create a new model with the modified architecture
new_model = keras.models.Model(inputs=base_model.input, outputs=output)

print(new_model.input)
print(new_model.output)

2023-05-14 13:44:18.609693: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


KerasTensor(type_spec=TensorSpec(shape=(None, 75, 75, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name=None), name='dense_1/Softmax:0', description="created by layer 'dense_1'")


In [6]:
new_model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=1e-5),
                  metrics=['accuracy'])

# Freezing the weights
for layer in base_model.layers[-20:]:
    layer.trainable = True
    
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 75, 75, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 37, 37, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 37, 37, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 37, 37, 32)   0           ['batch_normalization[0][0]']

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 7, 7, 64)    192         ['conv2d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_5 (Activation)      (None, 7, 7, 96)     0           ['batch_normalization_5[0][0]']  
                                                                                                  
 activation_7 (Activation)      (None, 7, 7, 64)     0           ['batch_normalization_7[0][0]']  
                                                                                                  
 activation_10 (Activation)     (None, 7, 7, 96)     0           ['batch_normalization_10[0][0]'] 
          

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_22 (BatchN  (None, 7, 7, 48)    144         ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_19 (Activation)     (None, 7, 7, 32)     0           ['batch_normalization_19[0][0]'] 
                                                                                                  
 activation_22 (Activation)     (None, 7, 7, 48)     0           ['batch_normalization_22[0][0]'] 
                                                                                                  
 conv2d_18 (Conv2D)             (None, 7, 7, 32)     10240       ['block35_1_ac[0][0]']           
          

 block35_3_mixed (Concatenate)  (None, 7, 7, 128)    0           ['activation_24[0][0]',          
                                                                  'activation_26[0][0]',          
                                                                  'activation_29[0][0]']          
                                                                                                  
 block35_3_conv (Conv2D)        (None, 7, 7, 320)    41280       ['block35_3_mixed[0][0]']        
                                                                                                  
 custom_scale_layer_2 (CustomSc  (None, 7, 7, 320)   0           ['block35_2_ac[0][0]',           
 aleLayer)                                                        'block35_3_conv[0][0]']         
                                                                                                  
 block35_3_ac (Activation)      (None, 7, 7, 320)    0           ['custom_scale_layer_2[0][0]']   
          

 activation_40 (Activation)     (None, 7, 7, 48)     0           ['batch_normalization_40[0][0]'] 
                                                                                                  
 conv2d_36 (Conv2D)             (None, 7, 7, 32)     10240       ['block35_4_ac[0][0]']           
                                                                                                  
 conv2d_38 (Conv2D)             (None, 7, 7, 32)     9216        ['activation_37[0][0]']          
                                                                                                  
 conv2d_41 (Conv2D)             (None, 7, 7, 64)     27648       ['activation_40[0][0]']          
                                                                                                  
 batch_normalization_36 (BatchN  (None, 7, 7, 32)    96          ['conv2d_36[0][0]']              
 ormalization)                                                                                    
          

...

                                                                                                  
 conv2d_163 (Conv2D)            (None, 1, 1, 192)    399360      ['mixed_7a[0][0]']               
                                                                                                  
 conv2d_166 (Conv2D)            (None, 1, 1, 256)    172032      ['activation_165[0][0]']         
                                                                                                  
 batch_normalization_163 (Batch  (None, 1, 1, 192)   576         ['conv2d_163[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_166 (Batch  (None, 1, 1, 256)   768         ['conv2d_166[0][0]']             
 Normalization)                                                                                   
          

                                                                                                  
 batch_normalization_174 (Batch  (None, 1, 1, 256)   768         ['conv2d_174[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_171 (Activation)    (None, 1, 1, 192)    0           ['batch_normalization_171[0][0]']
                                                                                                  
 activation_174 (Activation)    (None, 1, 1, 256)    0           ['batch_normalization_174[0][0]']
                                                                                                  
 block8_3_mixed (Concatenate)   (None, 1, 1, 448)    0           ['activation_171[0][0]',         
                                                                  'activation_174[0][0]']         
          

                                                                                                  
 block8_5_mixed (Concatenate)   (None, 1, 1, 448)    0           ['activation_179[0][0]',         
                                                                  'activation_182[0][0]']         
                                                                                                  
 block8_5_conv (Conv2D)         (None, 1, 1, 2080)   933920      ['block8_5_mixed[0][0]']         
                                                                                                  
 custom_scale_layer_34 (CustomS  (None, 1, 1, 2080)  0           ['block8_4_ac[0][0]',            
 caleLayer)                                                       'block8_5_conv[0][0]']          
                                                                                                  
 block8_5_ac (Activation)       (None, 1, 1, 2080)   0           ['custom_scale_layer_34[0][0]']  
          

 caleLayer)                                                       'block8_7_conv[0][0]']          
                                                                                                  
 block8_7_ac (Activation)       (None, 1, 1, 2080)   0           ['custom_scale_layer_36[0][0]']  
                                                                                                  
 conv2d_192 (Conv2D)            (None, 1, 1, 192)    399360      ['block8_7_ac[0][0]']            
                                                                                                  
 batch_normalization_192 (Batch  (None, 1, 1, 192)   576         ['conv2d_192[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_192 (Activation)    (None, 1, 1, 192)    0           ['batch_normalization_192[0][0]']
          

 Normalization)                                                                                   
                                                                                                  
 activation_200 (Activation)    (None, 1, 1, 192)    0           ['batch_normalization_200[0][0]']
                                                                                                  
 conv2d_201 (Conv2D)            (None, 1, 1, 224)    129024      ['activation_200[0][0]']         
                                                                                                  
 batch_normalization_201 (Batch  (None, 1, 1, 224)   672         ['conv2d_201[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_201 (Activation)    (None, 1, 1, 224)    0           ['batch_normalization_201[0][0]']
          

In [7]:
f1 = tfa.metrics.F1Score(num_classes=64, average='macro')

callbacks = keras.callbacks.EarlyStopping(monitor=f1, patience=3, mode='max', restore_best_weights=True)

new_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=1e-4), 
              metrics= [f1])


new_model.fit(train_data, epochs=100, callbacks=callbacks)

Epoch 1/100


2023-05-14 13:44:22.178377: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - ETA: 0s - loss: 0.5085 - f1_score: 0.0041WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 30s 534ms/step - loss: 0.5085 - f1_score: 0.0041
Epoch 2/100
4/4 [==============================] - ETA: 0s - loss: 0.2485 - f1_score: 0.0057WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 3s 848ms/step - loss: 0.2485 - f1_score: 0.0057
Epoch 3/100
4/4 [==============================] - ETA: 0s - loss: 0.1240 - f1_score: 0.0140WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


...

4/4 [==============================] - 2s 457ms/step - loss: 1.1487e-04 - f1_score: 1.0000
Epoch 97/100
4/4 [==============================] - ETA: 0s - loss: 1.1493e-04 - f1_score: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 2s 401ms/step - loss: 1.1493e-04 - f1_score: 1.0000
Epoch 98/100
4/4 [==============================] - ETA: 0s - loss: 1.1160e-04 - f1_score: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 2s 513ms/step - loss: 1.1160e-04 - f1_score: 1.0000
Epoch 99/100
4/4 [==============================] - ETA: 0s - loss: 1.1300e-04 - f1_score: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 2s 489ms/step - loss: 1.1300e-04 - f1_score: 1.0000
Epoch 100/100
4/4 [==============================] - ETA: 0s - loss: 1.1099e-04 - f1_score: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 2s 442ms/step - loss: 1.1099e-04 - f1_score: 1.0000


In [8]:
new_model.save('saved_model')

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets
